# Yelp Restaurant Analysis and Rating Prediction

## Team Members
Jun Hee Kim, Nikhil Rangarajan, Sander Shi

## Procedure
* Gather data from the Yelp API
* Extract and format features from the data
* Feature analysis and Variable selection
* Train using logistic regression
* Fine-tuning for logistic regression (regularization, etc.) 
* Train using neural network
* Fine-tuning for neural network (alteration of architecture, etc.)
* Model Selection using validation
* Final Analysis

## Part 0: Imports and Definitions of Constants

We will be using `pandas` to parse the data and `tensorflow` to construct the machine learning models.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

API_URL = "https://api.yelp.com/v3/businesses/search"

## Part 1: Data Gathering

In [2]:
def get_data():
    """
    This function loads all restaurant data from restaurants in Pittsburgh.
    """
    pass

all_restaurants_raw = get_data()

## Part 2: Feature Extraction

In [3]:
def extract_features(raw_data):
    """
    This function extracts the features from raw restaurant data.
    """
    pass

labeled_features = extract_features(all_restaurants_raw)

## Part 3: Logistic Regression

In [4]:
def train_logistic(labeled_data):
    """
    This function trains the features using logistic regression.
    """
    pass

theta = train_logistic(labeled_features)

## Part 4: Neural Network

In [5]:
def train_neural_net(labeled_data):
    """
    This function trains the features using a neural network.
    """
    pass

thetas_nn = train_neural_net(labeled_features)

## Part 5: Predictions and Testing Accuracy

In [6]:
def predict_logistic(features, trained):
    """
    This functions gives a prediction using logistic regression.
    """
    pass

def predict_nn(features, trained):
    """
    This function gives a prediction using the neural net.
    """
    pass

prediction_logistic = predict_logistic(_, theta)
prediction_nn = predict_nn(_, thetas_nn)

def report_accuracy_logistic(trained):
    pass

def report_accuracy_nn(trained):
    pass